In [1]:
suppressWarnings(suppressPackageStartupMessages({
    library(tidyverse)
    library(data.table)
}))


In [45]:
# input
idx_file="../gwas-current-gz-wc.20200629-131527.tsv"
check402_file='20200703-gwas-402.4.check.lst'

# output
check402_out_file='20200703-gwas-402.5.map.tsv'


In [3]:
check402_l <- fread(check402_file, head=F) %>% pull()

In [31]:
check402_df <- data.frame(
    f402=check402_l,
    stringsAsFactors=F
) %>%
separate(f402, c(rep(NA, 8), 'population', 'basename'), sep='/', remove=F, fill='right', extra='drop') %>%
separate(basename, c(NA, 'idx', NA, 'GBE_ID', NA), sep='\\.', remove=T, fill='right', extra='drop') %>%
mutate(GBE_ID = if_else(GBE_ID == 'PHENO1', idx, GBE_ID))


In [34]:
check402_df %>%
arrange(GBE_ID, population, f402) %>%
group_by(GBE_ID, population) %>%
summarise(f402=first(f402)) %>%
ungroup() -> check402_uniq_df


In [35]:
check402_uniq_df %>%
count(population)

population,n
<chr>,<int>
african,3378
e_asian,2885
non_british_white,3469
s_asian,3407
white_british,3764


In [36]:
idx_df <- fread(idx_file) %>%
rename('symlink'='#symlink')


In [37]:
idx_df %>%
filter(wc_l == 1080567) %>%
separate(symlink, c(rep(NA, 9), 'population', 'basename'), sep='/', remove=F, fill='right', extra='drop') %>%
separate(basename, c(NA, 'GBE_ID', NA), sep='\\.', remove=T, fill='right', extra='drop') %>%
left_join(
    check402_uniq_df, by=c('GBE_ID', 'population')
) -> check_df


In [38]:
check_df %>% 
filter(is.na(f402)) %>%
dim()

[1] 0 6

In [42]:
check_df %>% 
select(GBE_ID, population, symlink, location, f402) %>%
dim()

[1] 14246     5

In [46]:
check_df %>% 
select(GBE_ID, population, symlink, location, f402) %>%
rename('#GBE_ID' = 'GBE_ID') %>%
fwrite(check402_out_file, sep='\t', na = "NA", quote=F)
